# How to apply a custom perturbation to a given cross section

In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
import sandy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In this notebook we apply a constant relative perturbation, say 5%, to an energy-dependent cross section.
The perturbation is applied over an energy bin of choice, e.g. between 10 eV and 100 eV.

As an example we use the JEFF-3.3 evaluation for Pu-239  downloaded from the [OECD/NEA website](https://www.oecd-nea.org/dbdata/jeff/jeff33/index.html) and processed with NJOY-2016 into PENDF format using the following input file.

```
moder
20 -21 /
reconr
-21 -22 /
'sandy runs njoy'/
9437 0 0 /
0.001 0. /
0/
broadr
-21 -22 -23 /
9437 1 0 0 0. /
0.001 /
900.0 /
0 /
thermr
0 -23 -24 /
0 9437 20 1 1 0 0 1 221 0 /
900.0 /
0.001 0.1 /
moder
-24 30 /
```

The PENDF file in output `"tape30"` contains pointwise cross section for Pu-239 processed at 900 K.

### Read the file with sandy

A copy of the processed file is stored locally on this repository. 

In [ ]:
folder = "notebook_perturb_pu9_wims_grid"
filename = "tape30"
file = os.path.join(os.getcwd(), folder, filename)

Let's read the file with sandy and extract the cross sections.
In the process, `sandy` spits out a lot of warning saying the duplicate energy points are present... 

In [ ]:
tape = sandy.Endf6.from_file(file)
xs = sandy.Xs.from_endf6(tape)

Now, we apply perturbations according to the WIMS-69 energy grid, available on the [Serpent Wiki](http://serpent.vtt.fi/mediawiki/index.php/EPRI-CPM_69_group_structure).
Still, any other energy grids would work just fine.

In [ ]:
egrid = sandy.energy_grids.WIMS69
egrid

The energy grid was converted in eV to be consistent with the cross section data.

### Use a `sandy.Pert` object

SANDY uses a dedicated class to store (relative) perturbation coefficients.

In [ ]:
sandy.Pert([1, 1.05], index=[10, 100])

In this example the `Pert` instance defines a multiplicative perturbation factor 1 (0%) for all xs values between 0 and 10 eV, and 1.05 (5%) for all xs between 10 and 100 eV.

### Apply perturbations to cross sections

Below we apply perturbations to the fission cross section (MT=18) of Pu-239 (MAT=9437).
For each energy bin in the WIMS grid, we increase all xs points in that energy range by 30%.
The process is repreated iteratively and 69 perturbed xs objects are created.

In [ ]:
mat = 9437
mt = 18
pert_coeff = 30 / 100  # large perturbationfor better visualization

In [ ]:
perturbed_xs = []
for i in range(1, egrid.size):
    e_start = egrid[i - 1]
    e_stop = egrid[i]
    index = egrid[i - 1: i + 1]
    pert = sandy.Pert([1, 1 + pert_coeff], index=index)
    print(f"perturbed xs in energy bin #{i} [{e_start:.5e}, {e_stop:.5e}]")
    xspert = xs.custom_perturbation(mat, mt, pert)
    perturbed_xs.append(xspert)

### Plot the results

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
ax = xs.data[(mat,mt)].plot(logx=True, logy=True, color="dodgerblue", linewidth=2, ax=ax)
for xspert in perturbed_xs:
    ax = xspert.data[(mat,mt)].plot(logx=True, logy=True, alpha=0.5, linewidth=0.7, ax=ax)

ax.set_ylabel("cross section / b")
ax.set_xlabel("energy / eV")
ax.set_xlim([1e-5, 1e7])
ax.set_ylim([1e-1, 1e5])
fig.tight_layout();

### Write perturbed xs to file 

Finally, we can write each perturb cross section into a copy of the original PENDF file for further use.

In [ ]:
ipert = 0
tape_pert = perturbed_xs[ipert].to_endf6(tape)
text = tape_pert.to_string()

pert_file = os.path.join(os.getcwd(), folder, f"copy_pert{ipert}.pendf")
with open(pert_file, "w") as f:
    f.write(text)